In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d omkargurav/face-mask-dataset

 92% 150M/163M [00:01<00:00, 101MB/s] 
100% 163M/163M [00:01<00:00, 94.3MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/face-mask-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input,GlobalAveragePooling2D, Conv2D, MaxPooling2D,Flatten, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam,SGD
import glob
import tensorflow as tf
import shutil

In [ ]:
with_maskImage = os.listdir('/content/data/with_mask')
without_maskImage = os.listdir('/content/data/without_mask')

In [ ]:
os.mkdir('./train')
os.mkdir('./train/with_mask')
os.mkdir('./val')
os.mkdir('./val/with_mask')
os.mkdir('./test')
os.mkdir('./test/with_mask')


os.mkdir('./train/without_mask')
os.mkdir('./val/without_mask')
os.mkdir('./test/without_mask')

In [ ]:
with_mask_train_len = int(np.round(0.6 * len(with_maskImage), 0))
with_mask_val_len = int(np.round(0.7 * len(with_maskImage), 0))


for i in range(with_mask_train_len):
    shutil.copy(os.path.join('/content/data/with_mask', with_maskImage[i]), './train/with_mask')

for i in range(with_mask_train_len, with_mask_val_len):
    shutil.copy(os.path.join('/content/data/with_mask', with_maskImage[i]), './val/with_mask')

for i in range(with_mask_val_len, len(with_maskImage)):
    shutil.copy(os.path.join('/content/data/with_mask', with_maskImage[i]), './test/with_mask')


without_mask_train_len = int(np.round(0.6 * len(without_maskImage), 0))
without_mask_val_len = int(np.round(0.7 * len(without_maskImage), 0))



for i in range(without_mask_train_len):
    shutil.copy(os.path.join('/content/data/without_mask', without_maskImage[i]), './train/without_mask')

for i in range(without_mask_train_len, without_mask_val_len):
    shutil.copy(os.path.join('/content/data/without_mask', without_maskImage[i]), './val/without_mask')

for i in range(without_mask_val_len, len(without_maskImage)):
    shutil.copy(os.path.join('/content/data/without_mask', without_maskImage[i]), './test/without_mask')

In [ ]:
PATH_TRAIN_MASK = './train/with_mask'
PATH_VAL_MASK = './val/with_mask'
PATH_TEST_MASK = './test/with_mask'

PATH_TRAIN_NO_MASK = './train/without_mask'
PATH_VAL_NO_MASK = './val/without_mask'
PATH_TEST_NO_MASK = './test/without_mask'

PATH_TRAIN = './train'
PATH_VAL = './val'
PATH_TEST = './test'

In [ ]:
train_mask_files = os.listdir(PATH_TRAIN_MASK)
train_no_mask_files = os.listdir(PATH_TRAIN_NO_MASK)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

train_data_gen = ImageDataGenerator(rotation_range=30,
                                    width_shift_range=0.02,
                                    height_shift_range=0.02,
                                    zoom_range=[0.8,1.2],
                                    horizontal_flip=True,
                                    rescale=1/255
                                   )


val_data_gen = ImageDataGenerator(rescale=1/255)
test_data_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
training_data = train_data_gen.flow_from_directory(PATH_TRAIN,
                                                   target_size=(200, 200),
                                                   color_mode='rgb',
                                                   class_mode='binary',
                                                   batch_size=32)

val_data = val_data_gen.flow_from_directory(PATH_VAL,
                                            target_size=(200, 200),
                                            color_mode='rgb',
                                            class_mode='binary',
                                            batch_size=32)

test_data = test_data_gen.flow_from_directory(PATH_TEST,
                                              target_size=(200, 200),
                                              color_mode='rgb',
                                              class_mode='binary',
                                              batch_size=32)

Found 4532 images belonging to 2 classes.
Found 756 images belonging to 2 classes.
Found 2265 images belonging to 2 classes.


**RESNET50**

In [ ]:
from keras.applications.resnet50 import ResNet50

model=ResNet50(include_top=False,input_shape=(200, 200, 3))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 206, 206, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 100, 100, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 100, 100, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
inputl=Input(shape=(200, 200, 3), name='input')
l1=model(inputl)
l2=Flatten()(l1)
l4=Dropout(0.3)
l3=Dense(10,activation='relu')(l2)
l4=Dropout(0.3)
output=Dense(1,activation='sigmoid')(l3)
md=Model(inputs=[inputl],outputs=output)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=4, mode='min')

In [ ]:
md.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 200, 200, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 10)                1003530   
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 24591253 (93.81 MB)
Trainable params: 24538133 (93.61 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [ ]:
md.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
md.fit(training_data, batch_size=32, epochs=10, validation_data=val_data)

Epoch 1/10
 74/142 [==============>...............] - ETA: 34s - loss: 0.5943 - accuracy: 0.7441

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


142/142 [==============================] - 126s 530ms/step - loss: 0.4810 - accuracy: 0.7875 - val_loss: 3.3159 - val_accuracy: 0.4934
Epoch 2/10
142/142 [==============================] - 68s 478ms/step - loss: 0.4850 - accuracy: 0.7829 - val_loss: 0.6935 - val_accuracy: 0.5066
Epoch 3/10
142/142 [==============================] - 69s 481ms/step - loss: 0.2837 - accuracy: 0.8839 - val_loss: 0.6707 - val_accuracy: 0.5066
Epoch 4/10
142/142 [==============================] - 70s 493ms/step - loss: 0.2317 - accuracy: 0.8850 - val_loss: 0.8044 - val_accuracy: 0.5489
Epoch 5/10
142/142 [==============================] - 69s 484ms/step - loss: 0.2736 - accuracy: 0.9029 - val_loss: 0.4796 - val_accuracy: 0.8122
Epoch 6/10
142/142 [==============================] - 67s 472ms/step - loss: 0.2303 - accuracy: 0.9117 - val_loss: 0.5311 - val_accuracy: 0.7487
Epoch 7/10
142/142 [==============================] - 67s 471ms/step - loss: 0.1952 - accuracy: 0.9223 - val_loss: 0.1903 - val_accuracy: 0.

**VGG16**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

model_vgg=VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet")

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in model_vgg.layers:
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')

In [ ]:
last = model_vgg.layers[-1].output
x = Flatten()(last)
x = Dense(224, activation='softmax', name='predictions')(x)
vgg_model = Model(model_vgg.input, x)

In [ ]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
vgg_model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [ ]:
vgg_model.fit(training_data,validation_data=(val_data),epochs=10,verbose=1)

Epoch 1/10
142/142 [==============================] - 57s 400ms/step - loss: 0.2820 - accuracy: 0.9091 - val_loss: 0.2308 - val_accuracy: 0.9375
Epoch 2/10
142/142 [==============================] - 53s 373ms/step - loss: 0.2674 - accuracy: 0.9212 - val_loss: 0.2325 - val_accuracy: 0.9236
Epoch 3/10
142/142 [==============================] - 52s 366ms/step - loss: 0.2574 - accuracy: 0.9186 - val_loss: 0.1969 - val_accuracy: 0.9514
Epoch 4/10
142/142 [==============================] - 53s 376ms/step - loss: 0.2463 - accuracy: 0.9214 - val_loss: 0.1938 - val_accuracy: 0.9479
Epoch 5/10
142/142 [==============================] - 52s 367ms/step - loss: 0.2403 - accuracy: 0.9232 - val_loss: 0.1959 - val_accuracy: 0.9549
Epoch 6/10
142/142 [==============================] - 52s 369ms/step - loss: 0.2315 - accuracy: 0.9276 - val_loss: 0.1709 - val_accuracy: 0.9618
Epoch 7/10
142/142 [==============================] - 53s 376ms/step - loss: 0.2276 - accuracy: 0.9294 - val_loss: 0.2069 - val_ac

**INCEPTIONV3**

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

model_incep=InceptionV3(input_shape=(224,224,3),include_top=False,weights="imagenet")

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
model_incep.trainable=False
global_avg_pooling=GlobalAveragePooling2D()(model_incep.output)
prediction=Dense(1,activation="sigmoid")(global_avg_pooling)
inception_model=tf.keras.models.Model(inputs=model_incep.input,outputs=prediction)

In [ ]:
inception_model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")
inception_model.fit(training_data,validation_data=(val_data),epochs=10)

Epoch 1/10
142/142 [==============================] - 61s 388ms/step - loss: 0.1207 - accuracy: 0.9559 - val_loss: 0.0478 - val_accuracy: 0.9868
Epoch 2/10
142/142 [==============================] - 53s 375ms/step - loss: 0.0615 - accuracy: 0.9793 - val_loss: 0.0364 - val_accuracy: 0.9907
Epoch 3/10
142/142 [==============================] - 53s 373ms/step - loss: 0.0483 - accuracy: 0.9832 - val_loss: 0.0293 - val_accuracy: 0.9907
Epoch 4/10
142/142 [==============================] - 54s 382ms/step - loss: 0.0442 - accuracy: 0.9872 - val_loss: 0.0305 - val_accuracy: 0.9907
Epoch 5/10
142/142 [==============================] - 55s 386ms/step - loss: 0.0373 - accuracy: 0.9874 - val_loss: 0.0277 - val_accuracy: 0.9907
Epoch 6/10
142/142 [==============================] - 55s 385ms/step - loss: 0.0328 - accuracy: 0.9898 - val_loss: 0.0296 - val_accuracy: 0.9907
Epoch 7/10
142/142 [==============================] - 55s 389ms/step - loss: 0.0327 - accuracy: 0.9876 - val_loss: 0.0349 - val_ac

**DENSENET**

In [ ]:
base_model = tf.keras.applications.densenet.DenseNet201(weights='imagenet', input_shape = (224,224,3),
                                                     include_top=False)

Exception ignored in: <function _xla_gc_callback at 0x7ed97fee7010>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


74836368/74836368 [==============================] - 0s 0us/step


In [ ]:
from keras.models import Sequential, Model
base_model.trainable=False
model_dense=Sequential()
model_dense.add(base_model)
model_dense.add(GlobalAveragePooling2D())
model_dense.add(Flatten())
model_dense.add(Dense(300, activation="relu"))
model_dense.add(Dropout(0.5))
model_dense.add(Dense(100,activation='sigmoid'))
model_dense.add(Dropout(0.5))
model_dense.add(Dense(1,activation="relu"))
model_dense.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 6, 6, 1920)        18321984  
                                                                 
 global_average_pooling2d_3  (None, 1920)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_2 (Flatten)         (None, 1920)              0         
                                                                 
 dense_7 (Dense)             (None, 300)               576300    
                                                                 
 dropout_4 (Dropout)         (None, 300)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               30100     
                                                      

In [ ]:
model_dense.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [ ]:
model_dense.fit(training_data, validation_data = val_data, epochs = 10, verbose = 1)

Epoch 1/10
107/142 [=====================>........] - ETA: 13s - loss: 1.0577 - accuracy: 0.8778

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


142/142 [==============================] - 107s 442ms/step - loss: 1.0007 - accuracy: 0.8866 - val_loss: 0.0858 - val_accuracy: 0.9934
Epoch 2/10
142/142 [==============================] - 58s 407ms/step - loss: 0.7816 - accuracy: 0.9212 - val_loss: 0.0644 - val_accuracy: 0.9947
Epoch 3/10
142/142 [==============================] - 58s 410ms/step - loss: 0.4500 - accuracy: 0.9585 - val_loss: 0.0684 - val_accuracy: 0.9921
Epoch 4/10
142/142 [==============================] - 57s 404ms/step - loss: 0.3259 - accuracy: 0.9695 - val_loss: 0.1298 - val_accuracy: 0.9881
Epoch 5/10
142/142 [==============================] - 55s 387ms/step - loss: 0.3376 - accuracy: 0.9682 - val_loss: 0.0856 - val_accuracy: 0.9921
Epoch 6/10
142/142 [==============================] - 57s 401ms/step - loss: 0.3100 - accuracy: 0.9724 - val_loss: 0.0660 - val_accuracy: 0.9947
Epoch 7/10
142/142 [==============================] - 55s 390ms/step - loss: 0.2148 - accuracy: 0.9810 - val_loss: 0.0836 - val_accuracy: 0.